In [1]:
import pandas as pd
import numpy as np
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Activation, LayerNormalization, BatchNormalization
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.callbacks import EarlyStopping
from numpy.random import seed

In [2]:
import os
print(os.getcwd())
os.chdir(r"C:\Users\Nikhi\Desktop\Files\ML\final\final\Cross_section\csv")

C:\Users\Nikhi\OneDrive - purdue.edu\Mod 3\Machine Learning\Assignments\Project\Q1


In [3]:
Z_eff = pd.read_csv('data_Z_eff.csv',header=None)
R_eff = pd.read_csv('data_R_eff.csv',header = None)
R_org_eff = pd.read_csv('data_R_org_eff.csv',header = None)
group_ind = pd.read_csv('data_group_ind.csv',header =None)

In [4]:
Z_eff.shape

(2219316, 64)

In [5]:
t_train_start = list(range(0,41*12+1,12))
t_train_end =[x+120 for x in t_train_start]
t_val_start= [x for x in t_train_end]
t_val_end = [x+60 for x in t_val_start]
t_test_start = [x for x in t_val_end]
t_test_end = [x+12 for x in t_test_start]
t_test_end[41]=678

In [6]:
train_start = group_ind.iloc[t_train_start][0].to_list()
train_end = group_ind.iloc[t_train_end][0].to_list()
test_start = group_ind.iloc[t_test_start][0].to_list()
test_end = group_ind.iloc[t_test_end][0].to_list()
val_start = group_ind.iloc[t_val_start][0].to_list()
val_end = group_ind.iloc[t_val_end][0].to_list()                         

In [7]:
def portf_ret(preds, y_org):
    list_buy = [idx for idx, val in preds if val > np.percentile(preds,90)]
    list_buy = [idx for idx, val in preds if val < np.percentile(preds,10)]
    portf_ret_t = np.mean(y_true_t[list_buy]) - np.mean(y_true_t[list_sell])
    return portf_ret_t

In [8]:
y_pred = []
y_pred_val = []
y_pred_test = []
loss = []
model_list = []
y_org = []
y_org_val = []
y_org_test = []
y_reff = []
y_reff_val = []
y_reff_test = []
portf_ret_nn_test = []
R_pred=R_org_eff
seed(1)

In [ ]:
for retrain_idx in range(42):
    X = Z_eff.iloc[train_start[retrain_idx]:train_end[retrain_idx],:]
    X_idx = X.apply(pd.Series.nunique) != 1
    X = X.loc[:,X_idx]
    X_val = Z_eff.iloc[val_start[retrain_idx]:val_end[retrain_idx],:]
    X_test = Z_eff.iloc[test_start[retrain_idx]:test_end[retrain_idx],:]
    X_val = X_val.loc[:,X_idx]
    X_test = X_test.loc[:,X_idx]
    y =R_eff.iloc[train_start[retrain_idx]:train_end[retrain_idx],:]
    y_val = R_eff.iloc[val_start[retrain_idx]:val_end[retrain_idx],:]
    y_test = R_eff.iloc[test_start[retrain_idx]:test_end[retrain_idx],:]
    yorg = R_org_eff.iloc[train_start[retrain_idx]:train_end[retrain_idx],:]
    yorg_val = R_org_eff.iloc[val_start[retrain_idx]:val_end[retrain_idx],:]
    yorg_test=R_org_eff.iloc[test_start[retrain_idx]:test_end[retrain_idx],:]
    model = Sequential()
    model.add(LayerNormalization())
    model.add(Dense(64,activation='relu'))
    model.add(BatchNormalization())
    model.add(Dense(64,activation='relu'))
    model.add(BatchNormalization())
    model.add(Dense(32,activation='relu'))
    model.add(BatchNormalization())
    model.add(Dense(16,activation='relu'))
    model.add(BatchNormalization())
    model.add(Dense(8,activation='relu'))
    model.add(BatchNormalization())
    model.add(Dense(4,activation='relu'))
    model.add(BatchNormalization())
    model.add(Dense(1))
    model.compile(optimizer='adam',loss='mse')
    early_stop = EarlyStopping(monitor='val_loss', mode='min', verbose=1, patience=10)
    history = model.fit(x=X,y=y,validation_data=(X_val,y_val), batch_size=128, epochs=30,verbose=1,callbacks=[early_stop])
    preds = model.predict(X_test).flatten()
    y_org_test = yorg_test[0].to_list()
    R_pred.iloc[test_start[retrain_idx]:test_end[retrain_idx],:]=pd.DataFrame(preds)
    for t in range(t_test_start[retrain_idx], t_test_end[retrain_idx]):
        month_start = group_ind.iloc[t][0] - group_ind.iloc[t_test_start[retrain_idx]][0]
        month_end = group_ind.iloc[t+1][0] - group_ind.iloc[t_test_start[retrain_idx]][0]
        preds_month = preds[month_start:month_end]
        yorg_month = y_org_test[month_start:month_end]
        list_buy = [i for i, val in enumerate(preds_month) if val > np.quantile(preds_month,.9)]
        list_sell = [i for i, val in enumerate(preds_month) if val < np.quantile(preds_month,.1)]
        print(t,month_start,month_end,len(preds_month),len(list_buy),len(list_sell))
        portf_ret_nn_test.append(np.mean([yorg_month[i] for i in list_buy]) - np.mean([yorg_month[i] for i in list_sell]))
    loss.append(min(history.history['val_loss']))
    model_list.append(model)

Epoch 1/30
1606/1606 [==============================] - 10s 5ms/step - loss: 0.0234 - val_loss: 0.0102
Epoch 2/30
1606/1606 [==============================] - 8s 5ms/step - loss: 0.0091 - val_loss: 0.0097
Epoch 3/30
1606/1606 [==============================] - 11s 7ms/step - loss: 0.0088 - val_loss: 0.0095
Epoch 4/30
1606/1606 [==============================] - 17s 10ms/step - loss: 0.0087 - val_loss: 0.0097
Epoch 5/30
1606/1606 [==============================] - 17s 11ms/step - loss: 0.0086 - val_loss: 0.0095
Epoch 6/30
1606/1606 [==============================] - 17s 10ms/step - loss: 0.0085 - val_loss: 0.0098
Epoch 7/30
1606/1606 [==============================] - 15s 9ms/step - loss: 0.0085 - val_loss: 0.0098
Epoch 8/30
1606/1606 [==============================] - 11s 7ms/step - loss: 0.0084 - val_loss: 0.0102
Epoch 9/30
1606/1606 [==============================] - 10s 6ms/step - loss: 0.0084 - val_loss: 0.0104
Epoch 10/30
1606/1606 [==============================] - 11s 7ms/step -

2096/2096 [==============================] - 13s 6ms/step - loss: 0.0096 - val_loss: 0.0093
Epoch 8/30
2096/2096 [==============================] - 13s 6ms/step - loss: 0.0095 - val_loss: 0.0093
Epoch 9/30
2096/2096 [==============================] - 14s 7ms/step - loss: 0.0095 - val_loss: 0.0095
Epoch 10/30
2096/2096 [==============================] - 17s 8ms/step - loss: 0.0095 - val_loss: 0.0093
Epoch 11/30
2096/2096 [==============================] - 15s 7ms/step - loss: 0.0094 - val_loss: 0.0098
Epoch 12/30
2096/2096 [==============================] - 13s 6ms/step - loss: 0.0094 - val_loss: 0.0093
Epoch 13/30
2096/2096 [==============================] - 13s 6ms/step - loss: 0.0094 - val_loss: 0.0094
Epoch 14/30
2096/2096 [==============================] - 17s 8ms/step - loss: 0.0093 - val_loss: 0.0095
Epoch 15/30
2096/2096 [==============================] - 17s 8ms/step - loss: 0.0093 - val_loss: 0.0092
Epoch 16/30
2096/2096 [==============================] - 15s 7ms/step - loss: 

273 32577 36360 3783 379 379
274 36360 40211 3851 385 385
275 40211 44111 3900 390 390
Epoch 1/30
2579/2579 [==============================] - 24s 6ms/step - loss: 0.0274 - val_loss: 0.0097
Epoch 2/30
2579/2579 [==============================] - 14s 6ms/step - loss: 0.0096 - val_loss: 0.0094
Epoch 3/30
2579/2579 [==============================] - 16s 6ms/step - loss: 0.0094 - val_loss: 0.0093
Epoch 4/30
2579/2579 [==============================] - 20s 8ms/step - loss: 0.0093 - val_loss: 0.0093
Epoch 5/30
2579/2579 [==============================] - 18s 7ms/step - loss: 0.0092 - val_loss: 0.0093
Epoch 6/30
2579/2579 [==============================] - 19s 8ms/step - loss: 0.0091 - val_loss: 0.0093
Epoch 7/30
2579/2579 [==============================] - 23s 9ms/step - loss: 0.0091 - val_loss: 0.0094
Epoch 8/30
2579/2579 [==============================] - 16s 6ms/step - loss: 0.0090 - val_loss: 0.0093
Epoch 9/30
2579/2579 [==============================] - 17s 6ms/step - loss: 0.0090 - val

3271/3271 [==============================] - 23s 7ms/step - loss: 0.0096 - val_loss: 0.0090
Epoch 11/30
3271/3271 [==============================] - 21s 7ms/step - loss: 0.0096 - val_loss: 0.0090 0.0
Epoch 12/30
3271/3271 [==============================] - 19s 6ms/step - loss: 0.0095 - val_loss: 0.0089
Epoch 13/30
3271/3271 [==============================] - 21s 6ms/step - loss: 0.0095 - val_loss: 0.0091
Epoch 14/30
3271/3271 [==============================] - 21s 6ms/step - loss: 0.0095 - val_loss: 0.0089
Epoch 15/30
3271/3271 [==============================] - 22s 7ms/step - loss: 0.0095 - val_loss: 0.0089
Epoch 16/30
3271/3271 [==============================] - 20s 6ms/step - loss: 0.0095 - val_loss: 0.0090
Epoch 17/30
3271/3271 [==============================] - 23s 7ms/step - loss: 0.0095 - val_loss: 0.0089
Epoch 18/30
3271/3271 [==============================] - 25s 8ms/step - loss: 0.0095 - val_loss: 0.0089
Epoch 00018: early stopping
360 0 4404 4404 441 441
361 4404 8899 4495 4

In [11]:
np.nanmean(portf_ret_nn_test)/np.nanstd(portf_ret_nn_test)*np.sqrt(12)

5.245505721854921